# Предобработка данных

## Импорт библиотек

In [1]:
import pandas as pd
from PIL import Image
import pytesseract
import cv2
import os
import numpy as np

from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

## Загрузка и осмотр данных

In [2]:
shape_counter = 0
nan_counter = 0
columns = 10
df_list = []

for filename in os.listdir('405/'):
    if 'csv' in filename:
        path = '405/' + filename
        df = pd.read_csv(path)
        print('\n', filename)
        print(df.head())
        df_list.append(df)
        if df.shape[1] != columns:
            shape_counter += 1
        if df.isna().sum().sum() != 0:
            nan_counter += 1
print('\n', 'Число файлов где количество столбцов отличается от 10---', shape_counter)
print('\n', 'Число файлов с пропусками---', nan_counter)


 141899.csv
       ID  ID пользователя    Класс крови Дата донации  \
0   87330           141899  Цельная кровь   21.06.2019   
1  213527           141899         Плазма   29.08.2019   
2  213528           141899         Плазма   30.09.2019   
3  213529           141899         Плазма   01.11.2019   
4  213530           141899         Плазма   02.12.2019   

  Дата добавления донации   Тип донации                          Регион  \
0              18.11.2020  Безвозмездно  Россия, Татарстан, Альметьевск   
1              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
2              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
3              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
4              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   

   Место стадчи Статус донации Есть справка  
0           703        Принята           Да  
1           703        Принята           Да  
2           703        Принята           Да  
3  

**Чтож, неплохо. все таблицы имеют одинаковое число столбцов, пропусков нет. Жить можно**

## Добавляем город и убираем лишние столбцы

In [3]:
def region_to_city(st):
    return st.split(', ')[-1]

In [4]:
trash_columns = ['ID', 'ID пользователя', 'Дата добавления донации', 'Место стадчи', 'Статус донации', 'Есть справка', 'Регион']

for i in range(len(df_list)):
    df_list[i]['Город'] = df_list[i]['Регион'].apply(region_to_city)
    df_list[i] = df_list[i].drop(trash_columns, axis=1)

In [5]:
df_list[5].head()

,Класс крови,Дата донации,Тип донации,Город
0,Цельная кровь,03.07.2009,Безвозмездно,Ульяновск
1,Цельная кровь,16.09.2009,Безвозмездно,Ульяновск
2,Цельная кровь,16.03.2011,Безвозмездно,Ульяновск
3,Цельная кровь,01.08.2011,Безвозмездно,Ульяновск
4,Цельная кровь,24.11.2011,Безвозмездно,Ульяновск


## Функции для работы

### Обрезка изображений

В первом приближении - обрезаем пополам. Потому что у справки формы 405 таблица всегда в нижней половине.

In [6]:
def cut(img):
    height = int(img.shape[0])
    half = int(img.shape[0] / 2)
    return(img[half:height, :])

### Монохром

In [7]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

### Удаление шумов

In [8]:
def remove_noise(img):
    return cv2.medianBlur(img,1)

### Приведение к пороговому значению

In [9]:
#ret,thresh3 = cv.threshold(img,127,255,cv.THRESH_TRUNC)
#ret,thresh4 = cv.threshold(img,127,255,cv.THRESH_TOZERO)
def thresh(img):
    return cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,11)

### 

# Первые попытки что-то распознать

In [81]:
image_1 = cv2.imread('405/141899 .jpg')
image_1_pre = remove_noise(thresh((grayscale(cut(image_1)))))
cv2.imshow('after', (image_1_pre))
cv2.imshow('before', image_1)

cv2.waitKey()
cv2.destroyAllWindows()

#141899

Попытка найти слова и нарисовать вокруг них прямоугольники с помощью tesseract

In [79]:
d = pytesseract.image_to_data(image_1_pre, output_type=Output.DICT)

n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(image_1_pre, (x, y), (x + w, y + h), (255, 0, 0), 2)

cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()
#cv2.imwrite('iii.png', img)

Нахождение контуров таблиц с помощью open cv

In [97]:
contours, hierarchy = cv2.findContours(image_1_pre, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

img_contours = np.uint8(np.zeros((image_1_pre.shape[0],image_1_pre.shape[1])))

cv2.drawContours(img_contours, contours, -1, (255,255,255), 1)

cv2.imshow('preprocessed', image_1_pre)
cv2.imshow('boxed', img_contours)

cv2.waitKey()
cv2.destroyAllWindows()

In [107]:
cell = []
for contour in contours:
    if contour.shape[0] > 50:
        cell.append(contour)
    
#cl = np.array(cell).reshape((-1,1,2)).astype(np.int32)
img_contours_2 = np.uint8(np.zeros((image_1_pre.shape[0],image_1_pre.shape[1])))
cv2.drawContours(img_contours_2, cell, 0, (255,255,255), 1)
cv2.imshow('boxed', img_contours_2)
cv2.waitKey()
cv2.destroyAllWindows()

In [66]:
len(contours)

1686

In [41]:
string = pytesseract.image_to_string(image_1_pre, lang='rus')
print(string)

ПМ о си бу [“:

я’

"Отцавткый В взнтди ром (лаз, «лоток «ри, р.

Дата "Вид, `Кольво "Дата ‘ола. Н

а [# И ни ;

5 И И В В ИИ 4
Боваот5Р тиви [7 850 [05882025 | лев] |599 Гл | ивы | 9

Пре мввюр [бою [Або | подбево во чала Гален {98
поели [60 |авлооаЬ пав | 888 [0163382 | тии | 58
атое Г вмнибы | 660 [16372525 | лбу 600 15855282 м5) |280
ораезотв | омецбь) [ 880 [23932581] паб | 48 ЧтоБЯ паб 800
55012020] пяд16а) [806 _ [01082081 | пмацея [6% абБаваз ллаеу 808
ЭтОЗ2ОРО| пм {бе | 808 [25.08.2081 саму [ 800: [Ара тд |6
песо омдвы) | 608 [1607209 ВН 90 чаво сейв) | 889
1.055055 | `влвчвы 6007 | 16898 [ злё16 | 9® 5242023 [ мул8) | 90 `\
16572526| ма 464) | 500: | 1419201 туд (68) [60 И

летов Ает ны ОЙ бору


